##### Copyright 2018 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TPU の使用

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/guide/tpu"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">View on TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/tpu.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Run in Google Colab</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/tpu.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/tpu.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Download notebook</a></td>
</table>

Keras と Google Colab では現在、Cloud TPU を実験的にサポートしています。この Colab ノートブックをダウンロードする前に、Runtime > Change runtime type >  Hardware accelerator > TPU でノートブックの設定を確認し、ハードウェアアクセラレータが TPU であることを確認してください。

## セットアップ

In [ ]:
import tensorflow as tf

import os
import tensorflow_datasets as tfds

## TPU の初期化

TPU は通常、ユーザーの Python プログラムを実行するローカルプロセスとは異なる Cloud TPU ワーカー上にあります。そのため、リモートクラスタに接続して TPU を初期化するには、ある程度の初期化作業が必要となります。`TPUClusterResolver` の `tpu` 引数は、Colab だけの特別なアドレスであることに注意してください。Google Compute Engine（GCE）で実行している場合は、ご利用の CloudTPU の名前を渡す必要があります。

注意: TPU の初期化コードはプログラムのはじめにある必要があります。

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

## 手動でデバイスを配置する

TPU が初期されたら、計算を単一の TPU デバイスに配置するために、手動によるデバイスの配置を使用できます。


In [ ]:
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
with tf.device('/TPU:0'):
  c = tf.matmul(a, b)
print("c device: ", c.device)
print(c)

## 分散ストラテジー

ほとんどの場合、ユーザーはデータの並列化によって複数の TPU でモデルを実行することを望んでいます。分散ストラテジーは、モデルを CPU、GPU、または TPU で駆動するために使用できる抽象化です。分散ストラテジーを切り替えるだけで、モデルは特定のデバイスで実行するようになります。詳細は、「[分散ストラテジーガイド](./distributed_training.ipynb)」を参照してください。

最初に、`TPUStrategy` オブジェクトを作成します。

In [ ]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

計算を `strategy.run` API に渡すだけで、計算を複製してすべての TPU コアで実行できるようにすることができます。次のコード例では、すべてのコアが同じ入力 `(a, b)` を取得し、各コアで独立して matmul を実行しています。出力は、すべてのレプリカからの値となります。

In [ ]:
@tf.function
def matmul_fn(x, y):
  z = tf.matmul(x, y)
  return z

z = strategy.run(matmul_fn, args=(a, b))
print(z)

## TPU での分類

基本的な概念を学んだので、より具体的な例を見てみることにしましょう。このガイドでは、分散ストラテジー `tf.distribute.experimental.TPUStrategy` を使用して Cloud TPU を駆動し、Keras モデルをトレーニングする方法を説明します。


### Keras モデルを定義する

次のコードは、Keras を使用した MNIST モデルの定義です。CPU または GPU で使用されるものと変わりません。変数が各 TPU デバイスに作成されるように、Keras モデルの作成を `strategy.scope` 内で行う必要があることに注意してください。ほかの部分のコードは、ストラテジーのスコープ内でなくても構いません。

In [ ]:
def create_model():
  return tf.keras.Sequential(
      [tf.keras.layers.Conv2D(256, 3, activation='relu', input_shape=(28, 28, 1)),
       tf.keras.layers.Conv2D(256, 3, activation='relu'),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(256, activation='relu'),
       tf.keras.layers.Dense(128, activation='relu'),
       tf.keras.layers.Dense(10)])

### 入力データセット

Cloud TPU にデータを迅速にフィードできなければ Cloud TPU を使用することは不可能であるため、Cloud TPU を使用する際は、`tf.data.Dataset` API を効率的に使用できることが非常に重要となります。データセットのパフォーマンスに関する詳細は、ｐ[入力パイプラインのパフォーマンスガイド](./data_performance.ipynb)」を参照してください。

最も単純な実験（`tf.data.Dataset.from_tensor_slices` またはほかのグラフ内データの使用）以外のすべての実験では、データセットが読み取るすべてのデータファイルを Google Cloud Storage（GCS）バケットに格納する必要があります。

ほとんどの使用事例では、データを `TFRecord` 形式に変換し、`tf.data.TFRecordDataset` を使って読み取ることをお勧めします。このやり方については、「[TFRecord および tf.Example のチュートリアル](../tutorials/load_data/tfrecord.ipynb)」を参照してください。ただし、これは絶対要件ではないため、ほかのデータセットリーダー（`FixedLengthRecordDataset` または `TextLineDataset`）を使用することもできます。

小さなデータセットは、`tf.data.Dataset.cache` を使ってすべてをメモリに読み込むことができます。

データ形式にかかわらず、100 MB 程度の大きなファイルを使用することをお勧めします。このネットワーク化された設定においては、ファイルを開くタスクのオーバーヘッドが著しく高くなるため、特に重要なことです。

ここでは、`tensorflow_datasets` モジュールをし米須ヒデ、MNIST トレーニングデータのコピーを取得する必要があります。`try_gcs` は、パブリック GCS バケットで提供されているコピーを使用するように指定されています。 これを指定しない場合、TPUはダウンロードされたデータにアクセスできません。 

In [ ]:
def get_dataset(batch_size, is_training=True):
  split = 'train' if is_training else 'test'
  dataset, info = tfds.load(name='mnist', split=split, with_info=True,
                            as_supervised=True, try_gcs=True)

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0

    return image, label

  dataset = dataset.map(scale)

  # Only shuffle and repeat the dataset in training. The advantage to have a
  # infinite dataset for training is to avoid the potential last partial batch
  # in each epoch, so users don't need to think about scaling the gradients
  # based on the actual batch size.
  if is_training:
    dataset = dataset.shuffle(10000)
    dataset = dataset.repeat()

  dataset = dataset.batch(batch_size)

  return dataset

### Keras の高位 API を使用してモデルをトレーニングする

Keras の fit/compile API を使用するだけでモデルをトレーニングできます。ここでは、TPU 固有のものはないため、複数の GPU を使用している場合や、`TPUStrategy` ではなく`MirroredStrategy` を使用している場合でも、以下のコードを記述できます。詳細については、「[Kera wo使用した分散トレーニング](https://www.tensorflow.org/tutorials/distribute/keras)」チュートリアルを参照してください。

In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

batch_size = 200
steps_per_epoch = 60000 // batch_size
validation_steps = 10000 // batch_size

train_dataset = get_dataset(batch_size, is_training=True)
test_dataset = get_dataset(batch_size, is_training=False)

model.fit(train_dataset,
          epochs=5,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset, 
          validation_steps=validation_steps)

Python のオーバーヘッドを緩和し、TPU のパフォーマンスを最大化するには、`Model.compile` の **実験的な** `experimental_steps_per_execution` 引数を使用してみてください。スループットが約 50% 改善されます。

In [ ]:
with strategy.scope():
  model = create_model()
  model.compile(optimizer='adam',
                # Anything between 2 and `steps_per_epoch` could help here.
                experimental_steps_per_execution = 50,
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['sparse_categorical_accuracy'])

model.fit(train_dataset,
          epochs=5,
          steps_per_epoch=steps_per_epoch,
          validation_data=test_dataset,
          validation_steps=validation_steps)

### カスタムトレーニングループを使用してモデルをトレーニングする

`tf.function` と `tf.distribute` API を直接使用しても、モデルを作成し、トレーニングすることができます。`strategy.experimental_distribute_datasets_from_function` API は、データセット関数を指定してデータセットを配布するために使用されます。この場合、データセットに渡されるバッチサイズは、グローバルバッチサイズではなく、レプリカごとのバッチサイズであることに注意してください。詳細については、「[tf.distribute.Strategy によるカスタムトレーニング](https://www.tensorflow.org/tutorials/distribute/custom_training)」チュートリアルを参照してください。


最初に、モデル、データセット、および tf.functions を作成します。

In [ ]:
# Create the model, optimizer and metrics inside strategy scope, so that the
# variables can be mirrored on each device.
with strategy.scope():
  model = create_model()
  optimizer = tf.keras.optimizers.Adam()
  training_loss = tf.keras.metrics.Mean('training_loss', dtype=tf.float32)
  training_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
      'training_accuracy', dtype=tf.float32)

# Calculate per replica batch size, and distribute the datasets on each TPU
# worker.
per_replica_batch_size = batch_size // strategy.num_replicas_in_sync

train_dataset = strategy.experimental_distribute_datasets_from_function(
    lambda _: get_dataset(per_replica_batch_size, is_training=True))

@tf.function
def train_step(iterator):
  """The step function for one training step"""

  def step_fn(inputs):
    """The computation to run on each TPU device."""
    images, labels = inputs
    with tf.GradientTape() as tape:
      logits = model(images, training=True)
      loss = tf.keras.losses.sparse_categorical_crossentropy(
          labels, logits, from_logits=True)
      loss = tf.nn.compute_average_loss(loss, global_batch_size=batch_size)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)
    training_accuracy.update_state(labels, logits)

  strategy.run(step_fn, args=(next(iterator),))

次に、トレーニングループを実行します。

In [ ]:
steps_per_eval = 10000 // batch_size

train_iterator = iter(train_dataset)
for epoch in range(5):
  print('Epoch: {}/5'.format(epoch))

  for step in range(steps_per_epoch):
    train_step(train_iterator)
  print('Current step: {}, training loss: {}, accuracy: {}%'.format(
      optimizer.iterations.numpy(),
      round(float(training_loss.result()), 4),
      round(float(training_accuracy.result()) * 100, 2)))
  training_loss.reset_states()
  training_accuracy.reset_states()

### `tf.function` 内の複数のステップでパフォーマンスを改善する

`tf.function` 内で複数のステップを実行することで、パフォーマンスを改善できます。これは、`tf.function` 内の `tf.range` で `strategy.run` 呼び出しをラッピングすることで実現されます。AutoGraph は、TPU ワーカーの `tf.while_loop` に変換します。

パフォーマンスは改善されますが、`tf.function` 内の単一のステップに比べれば、トレードオフがあります。`tf.function` で複数のステップを実行すると柔軟性に劣り、ステップ内での Eager execution や任意の Python コードを実行できません。


In [ ]:
@tf.function
def train_multiple_steps(iterator, steps):
  """The step function for one training step"""

  def step_fn(inputs):
    """The computation to run on each TPU device."""
    images, labels = inputs
    with tf.GradientTape() as tape:
      logits = model(images, training=True)
      loss = tf.keras.losses.sparse_categorical_crossentropy(
          labels, logits, from_logits=True)
      loss = tf.nn.compute_average_loss(loss, global_batch_size=batch_size)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(list(zip(grads, model.trainable_variables)))
    training_loss.update_state(loss * strategy.num_replicas_in_sync)
    training_accuracy.update_state(labels, logits)

  for _ in tf.range(steps):
    strategy.run(step_fn, args=(next(iterator),))

# Convert `steps_per_epoch` to `tf.Tensor` so the `tf.function` won't get 
# retraced if the value changes.
train_multiple_steps(train_iterator, tf.convert_to_tensor(steps_per_epoch))

print('Current step: {}, training loss: {}, accuracy: {}%'.format(
      optimizer.iterations.numpy(),
      round(float(training_loss.result()), 4),
      round(float(training_accuracy.result()) * 100, 2)))

## 次のステップ

- [Google Cloud TPU ドキュメント](https://cloud.google.com/tpu/docs/) - Google Cloud TPU をセットアップして実行します。
- [TensorFlow での分散型トレーニング](./distributed_training.ipynb) - 分散ストラテジーの使用方法とベストプラクティスを示す多数の例へのリンクが含まれます。
- [TensorFlow でモデルを保存/読み込む](https://github.com/tensorflow/models/tree/master/official) - 分散ストラテジーでモデルを保存および読み込む方法を説明します。
- [TensorFlow 公式モデル](https://cloud.google.com/tpu/docs/performance-guide) - Cloud TPU 対応の最新の TensorFlow 2.x モデルの例を紹介します。
- [The Google Cloud TPU Performance Guide](https://cloud.google.com/tpu/docs/performance-guide) - Enhance Cloud TPU performance further by adjusting Cloud TPU configuration parameters for your application.